In [ ]:
import matplotlib
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import os
import umap
import datashader as ds
import colorcet as cc
import igraph
import tqdm
from scipy import sparse
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans

from matplotlib.collections import PolyCollection
from matplotlib.colors import ListedColormap

from dredFISH.Analysis import TissueGraph
from dredFISH.Visualization import Viz
from dredFISH.Utils.__init__plots import * 
from dredFISH.Utils import powerplots
from dredFISH.Utils import miscu
from dredFISH.Utils import imageu
from dredFISH.Utils import tmgu
from dredFISH.Utils import AmbrosioTortorelliMinimizer

import importlib
importlib.reload(Viz)
importlib.reload(TissueGraph)
importlib.reload(imageu)

https://github.com/jacobgil/Ambrosio-Tortorelli-Minimizer

In [ ]:
img = np.random.randn(50,50)
for i in range(50):
    for j in range(50):
        img[i,j] += 10*((i-20)**2+(j-20)**2>15**2)
plt.imshow(img)

In [ ]:
# img = np.random.randn# a matrix
solver = AmbrosioTortorelliMinimizer.AmbrosioTortorelliMinimizer(img, iterations=300)
img2, edges = solver.minimize()

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(3*4,1*4))
ax = axs[0]
ax.imshow(img)
ax.set_title('orig')
ax = axs[1]
ax.imshow(img2)
ax.set_title('model')
ax = axs[2]
ax.imshow(edges)
ax.set_title('edges')
plt.show()

In [ ]:
sns.histplot(img.reshape(-1,))

In [ ]:
sns.histplot(img2.reshape(-1,))

In [ ]:
sns.histplot(edges.reshape(-1,))

In [ ]:
edges

# Can this be extended to multi-channel problem? 
Yes:
```python
	img = cv2.imread(sys.argv[1], 1)
	result, edges = [], []
	for channel in cv2.split(img):
		solver = AmbrosioTortorelliMinimizer(channel, iterations = 1, tol = 0.1, solver_maxiterations = 6)
		f, v = solver.minimize()
		result.append(f)
		edges.append(v)

	f = cv2.merge(result) # concatenate channels
	v = np.maximum(*edges) # get the maximum edges
```

#### Load data

In [ ]:
respath = '/bigstore/GeneralStorage/fangming/projects/dredfish/figures/'

In [ ]:
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t5'
!ls -alhtr $basepth
!head $basepth"/TMG.json"

In [ ]:
df = pd.read_csv(
    os.path.join(basepth, "default_analysis.csv"))
df

In [ ]:
TMG = TissueGraph.TissueMultiGraph(basepath=basepth, 
                                   redo=False, # load existing 
                                  )

In [ ]:
# spatial coordinates
layer = TMG.Layers[0]
XY = layer.XY
x, y = XY[:,0], XY[:,1]

cells = layer.adata.obs.index.values

N = layer.N
# measured basis
ftrs_mat = layer.feature_mat

# umap_mat = umap.UMAP(n_neighbors=30, min_dist=0.1).fit_transform(ftrs_mat)

# types
types = df['type_r0.1'].values
# local env
env_mat = tmgu.get_local_type_abundance(types, SG=layer.SG)

In [ ]:
env_mat

In [ ]:
tmp = pd.DataFrame(np.hstack([XY, env_mat])).rename({0:'x', 1:'y'}, axis=1)
tmp

In [ ]:
imgmat = imageu.valued_pointset_to_image(np.hstack([XY, env_mat[:,[0]]]), resolution=100)
imgmat.shape

In [ ]:
plt.imshow(imgmat, cmap='rocket_r')

In [ ]:
# result, edges = [], []
# for channel in cv2.split(img):
#     solver = AmbrosioTortorelliMinimizer(channel, iterations = 1, tol = 0.1, solver_maxiterations = 6)
#     f, v = solver.minimize()
#     result.append(f)
#     edges.append(v)
    
# f = cv2.merge(result)
# v = np.maximum(*edges)

In [ ]:
imgs, models, alledges = [], [], []
for i in range(env_mat.shape[1]):
    imgmat = imageu.valued_pointset_to_image(np.hstack([XY, env_mat[:,[i]]]), resolution=50)
    channel = np.nan_to_num(imgmat, 0) #.fillna(0)
    solver = AmbrosioTortorelliMinimizer.AmbrosioTortorelliMinimizer(channel, iterations=1000, tol=0.1,)
    model, edges = solver.minimize()
    imgs.append(channel)
    models.append(model)
    alledges.append(edges)

    fig, axs = plt.subplots(1,3,figsize=(3*4,1*4))
    ax = axs[0]
    ax.imshow(channel, origin='lower', aspect='equal',)
    ax.set_title('orig')
    ax.axis('off')
    ax = axs[1]
    ax.imshow(model, origin='lower', aspect='equal',)
    ax.set_title('model')
    ax.axis('off')
    ax = axs[2]
    ax.imshow(edges, origin='lower', aspect='equal',)
    ax.set_title('edges')
    ax.axis('off')
    fig.subplots_adjust(wspace=0.01)
    plt.show()

In [ ]:
combimg = np.argmax(np.stack(imgs), axis=0)
combmodel = np.argmax(np.stack(models), axis=0)
combedges = np.maximum.reduce(alledges)

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(3*10,1*10))
ax = axs[0]
ax.imshow(combimg, origin='lower', aspect='equal',)
ax.set_title('orig')
ax.axis('off')

ax = axs[1]
ax.imshow(combmodel, origin='lower', aspect='equal',)
ax.set_title('model')
ax.axis('off')

ax = axs[2]
ax.imshow(combedges, origin='lower', aspect='equal',)
ax.set_title('edges')
ax.axis('off')
fig.subplots_adjust(wspace=0.01)
plt.show()

In [ ]:
n = len(imgs)
fig, axss = plt.subplots(n,3,figsize=(3*4,n*4))
for axs, channel, model, edges in zip(axss, imgs, models, alledges):
    ax = axs[0]
    ax.imshow(channel, origin='lower', aspect='equal',)
    ax.set_title('orig')
    ax.axis('off')
    ax = axs[1]
    ax.imshow(model, origin='lower', aspect='equal',)
    ax.set_title('model')
    ax.axis('off')
    ax = axs[2]
    ax.imshow(edges, origin='lower', aspect='equal',)
    ax.set_title('edges')
    ax.axis('off')
    fig.subplots_adjust(wspace=0.01)
plt.show()

In [ ]:
# improvement -- to add the hull -- np.nan from 0
# parameters to tune?
# problem: this turns cell type abundance into cell type dominance (not necessarily the best we want -- we want unique combinations of cell type abundances)